In [9]:
from nvsys import *
!jupyter nbconvert --to script nvbloch.ipynb

[NbConvertApp] Converting notebook nvbloch.ipynb to script
[NbConvertApp] Writing 1574 bytes to nvbloch.py


In [4]:
def nvblochvecs(states, proj0, proj1, projx, times, H_lab, sphere=True):
    #------
    # Returns bloch vectors
    #------
    # states - an array of column vectors
    # proj0, proj1 and projx must be row vectors
    # times - an array of times
    # H_lab - square matrix. For conversion into rotating frame.
    #
    #------
    if sphere:
        factor_sphere = 2
    else:
        factor_sphere = 1
        
    opR = [(1j*H_lab*time_loop).expm() for time_loop in times]
    states_rot = [opR[idx]*states[idx] for idx in np.arange(len(times))]

    # Convert complex coefficients to polar form 
    c0 = [(proj0*elem)[0][0][0] for elem in states_rot]
    r0, phase0 = np.absolute(c0), np.angle(c0)
    c1 = [(proj1*elem)[0][0][0] for elem in states_rot]
    r1, phase1 = np.absolute(c1), np.angle(c1)

    # Find magntiude of coefficient of other states, for normalization
    cx = [(projx*elem)[0][0][0] for elem in states_rot]
    rx = np.absolute(cx)

    # Calculate 3D polar and azimuthal angles
    angle_polar1 = factor_sphere*np.arcsin(r1/((1 - rx**2)**0.5))
    angle_polar2 = factor_sphere*np.arccos(r0/((1 - rx**2)**0.5))
    angle_polar3 = factor_sphere*np.arctan2(r1, r0)
    angle_polar = angle_polar1
    angle_azi = phase1 - phase0

    xb = np.sin(angle_polar)*np.cos(angle_azi)
    yb = np.sin(angle_polar)*np.sin(angle_azi)
    zb = np.cos(angle_polar)
    
    return xb, yb, zb